#
  - 3. Syndigo Mapping ML dict.ip
  - Bsse on the code from offshore and modifcation with
    - Construct a dictionary with key: 'Level 1' value as file name
  - 3. Syndigo Mapping MachineLearning

In [111]:
def unique_no_nan(x):
    return x.dropna().unique()

In [143]:
def constrcut_dic():
    key = unique_no_nan(syndigo_mapped['Level 1'])
    value = list( map(lambda x : re.sub('&|/', '', x), key))
    value = list( map(lambda x : re.sub(' +', '_', x), value))
    value = list(map(str.lower, value))
    #value = ['-'.join(['tfidf'+ str]), value]

    print(f'key: {key}') 
    print(f'value: {value}')
    return dict(zip(key, value)) 

In [144]:
dic_level1= constrcut_dic()
dic_level1

key: ['Food / Beverages' 'Health & Beauty' 'Beer / Wine / Spirits'
 'Kitchen & Bathroom' 'Cleaning & Janitorial' 'Home & Venue Decoration'
 'Livestock & Pet Supplies' 'Toys / Games / Hobbies' 'Childcare'
 'Tobacco Products' 'Apparel' 'Gardening & Outdoors'
 'Sports & Outdoor Recreation Equipment' 'Office Supplies'
 'Plumbing & Water Service' 'Lighting & Fans' 'Appliances' 'Electronics'
 'Not classified' 'Automotive' 'Tools' 'Hardware' 'Books & Videos'
 'Arts & Crafts' 'Electrical' 'Heating / Ventilation / Air Conditioning'
 'Hospitality Supplies' 'Furniture' 'Power Sports'
 'Power Transmission & Motion Control' 'Marine' 'Flooring'
 'Paints & Coatings' 'Building Supplies' 'Musical Instruments'
 'Material Handling' 'Agricultural Equipment']
value: ['food_beverages', 'health_beauty', 'beer_wine_spirits', 'kitchen_bathroom', 'cleaning_janitorial', 'home_venue_decoration', 'livestock_pet_supplies', 'toys_games_hobbies', 'childcare', 'tobacco_products', 'apparel', 'gardening_outdoors', 'spor

{'Food / Beverages': 'food_beverages',
 'Health & Beauty': 'health_beauty',
 'Beer / Wine / Spirits': 'beer_wine_spirits',
 'Kitchen & Bathroom': 'kitchen_bathroom',
 'Cleaning & Janitorial': 'cleaning_janitorial',
 'Home & Venue Decoration': 'home_venue_decoration',
 'Livestock & Pet Supplies': 'livestock_pet_supplies',
 'Toys / Games / Hobbies': 'toys_games_hobbies',
 'Childcare': 'childcare',
 'Tobacco Products': 'tobacco_products',
 'Apparel': 'apparel',
 'Gardening & Outdoors': 'gardening_outdoors',
 'Sports & Outdoor Recreation Equipment': 'sports_outdoor_recreation_equipment',
 'Office Supplies': 'office_supplies',
 'Plumbing & Water Service': 'plumbing_water_service',
 'Lighting & Fans': 'lighting_fans',
 'Appliances': 'appliances',
 'Electronics': 'electronics',
 'Not classified': 'not_classified',
 'Automotive': 'automotive',
 'Tools': 'tools',
 'Hardware': 'hardware',
 'Books & Videos': 'books_videos',
 'Arts & Crafts': 'arts_crafts',
 'Electrical': 'electrical',
 'Heating /

In [100]:
def prepare_data_1():
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = True)}")    
    return syndigo_mapped, pim_gtin_mapped

In [89]:
def prepare_data_2(level__1):
    level_1 = level__1
    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
    print(f'1. type(subset_df)= {type(subset_df)}, len(subset_df)= {len(subset_df)} subset_df.shape {subset_df.shape}  \n subset_df.head {subset_df.head()}') 
    series = syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)
    print(f' series {series} \n type(series)= {type(series)}, len(series)= {len(series)} \n series.tolist() {series.tolist()} ')      
    if series.tolist():
        subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series==1].index.tolist())].index)
    print(f'2. type(subset_df)= {type(subset_df)}, len(subset_df)= {len(subset_df)} subset_df.shape {subset_df.shape}  \n subset_df.head {subset_df.head()}')
    
    vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 50000)
    X_subset = vect_subset.fit_transform(subset_df.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  X_subset, y_subset, subset_df, level2_id_map

In [18]:
def prep_data_3(X_subset, subset_df):
    dfx = pd.DataFrame(X_subset.toarray())
    subset_df[['Level 2']]
    dfy =  subset_df['Level 2']
    df1=dfx.reset_index().drop('index', axis = 1) 
    df2=dfy.reset_index().drop('index', axis = 1)
    df3 = pd.merge(df1, df2, left_index=True, right_index=True)
    return df3

In [29]:
def proc_predict(level__1):
     global A_train, A_test, B_train, B_test, X_train, X_test, y_train, y_test
     A_train, A_test, B_train, B_test = train_test_split(subset_df.GTIN.tolist(), subset_df['Level 2'].tolist(), test_size=0.1, stratify=y_subset, random_state=42)
     X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size= 0.1, stratify=y_subset, random_state=42)
     lr_tf = LogisticRegressionCV(cv = 5, multi_class = 'multinomial', solver = 'saga', n_jobs=-1, max_iter=10000000)
     
     print(f"Training - \"{level__1}\"")
     lr_tf.fit(X_train, y_train)
     preds = lr_tf.predict(X_test)
     print(f"type(preds) {type(preds)} len(preds) {len(preds)} \n preds[0:10] {preds[0:10]}")
     preds_lrtf = preds
     probs = lr_tf.predict_proba(X_test)
     print(f"type(probs) {type(probs)} len(probs) {len(probs)} \n probs[0:10] {probs[0:10]}")   
     preds_train = lr_tf.predict(X_train)
     print(f"type(preds_train) {type(preds_train)} len(preds_train) {len(preds_train)} \n preds_train[0:10] {preds_train[0:10]} ")   
     probs_train = lr_tf.predict_proba(X_train)
     print(f"type(probs_train) {type(probs_train)} len(probs_train) {len(probs_train)} \n probs_train[0:10] {probs_train[0:10]}")   
     print(f"Done Training - \"{level__1}\"")
     return preds, preds_lrtf, probs, preds_train, probs_train  

In [95]:
def proc_append_2():
    
    test_new_proobs = []
    
    print(f'len(preds)  {len(preds)}')
    for i in range(len(preds)):
        test_new_proobs.append(probs[i][np.argsort(probs[i])][::-1][:1].tolist())
    test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]
    print(f'test_new_proobs {test_new_proobs}')
    
    train_new_proobs = []
    
    print(f'len(preds_train)  {len(preds_train)}')
    for i in range(len(preds_train)):
         test_new_proobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1].tolist())
    train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]
    print(f'train_new_proobs {train_new_proobs}')  
    
    # process testLevels & trainLevelss 
    testLevels = []
    for j in y_test:
        testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    print(f'testLevels {testLevels [0:10]}')
    testLevelss = []
    for j in preds:
        testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    print(f'testLevelss {testLevelss [0:10]}')
    
    trainLevelss = []
    for j in preds_train:
        trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    
    return  test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss

In [96]:
 test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss = proc_append_2()

len(preds)  50
test_new_proobs [0.9999645691689025, 0.9090362378545995, 0.9708339421543606, 0.999461348460541, 0.9982907479934469, 0.9999201160992794, 0.6774044858548016, 0.9917925948320767, 0.5678791544528573, 0.9999222411324367, 0.9708068517296905, 0.9982535757918141, 0.999659682416689, 0.9993190018680612, 0.999666315420207, 0.988626487089105, 0.9999883223277677, 0.9998223107870962, 0.9999828696781584, 0.9998910494283926, 0.9999502611537154, 0.999990804658139, 0.9999825977462176, 0.999714271103653, 0.9998451308841866, 0.9997880318472873, 0.9999464699797422, 0.9971400447291249, 0.5464251071098462, 0.9982078630402004, 0.895537543110456, 0.9998090985798066, 0.9992607025351912, 0.9951540822240846, 0.993233539261957, 0.9999278500632993, 0.8810342653807214, 0.9992812260689606, 0.9999900664579909, 0.9980674916247813, 0.9995549760108314, 0.998650719220997, 0.9340107502192513, 0.9880056191898343, 0.9964819009567214, 0.9881204276546408, 0.9563998732840001, 0.9994197692865763, 0.999888831630773

In [69]:
def proc_append_1():
    level_num = []
    testProobs = []
    number = 1
    
    print(f'len(preds)  {len(preds)}')
    for i in range(len(preds)):
        #for j in range(number):
        #    
        #    try:
        #        level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]))
        #        print(f'1. i = {i} j = {j} level_num {level_num}')
        #    except TypeError:
        #        level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]))
        #        print("problem occurred with: ", i, j, flush=True)
        testProobs.append(probs[i][np.argsort(probs[i])][::-1][:1])
        print(f'2. i = {i}  testProobs {testProobs}')
    
    print(f'len(testProobs)  {len(testProobs)}')
    test_new_proobs = []
    for i in range(len(testProobs)):
        test_new_proobs.append(testProobs[i].tolist())
    test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]
    print(f'test_new_proobs {test_new_proobs}')
    return

In [70]:
proc_append_1()

len(preds)  50
2. i = 0  testProobs [array([0.99996457])]
2. i = 1  testProobs [array([0.99996457]), array([0.90898802])]
2. i = 2  testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784])]
2. i = 3  testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172])]
2. i = 4  testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172]), array([0.99828899])]
2. i = 5  testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172]), array([0.99828899]), array([0.9999203])]
2. i = 6  testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172]), array([0.99828899]), array([0.9999203]), array([0.67778319])]
2. i = 7  testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172]), array([0.99828899]), array([0.9999203]), array([0.67778319]), array([0.99179779])]
2. i = 8  testProobs [array([0.99996457]), array([0.9089

In [67]:
def proc_append():
    level_num = []
    testProobs = []
    number = 1
    
    print(f'len(preds)  {len(preds)}')
    for i in range(len(preds)):
        for j in range(number):
            
            try:
                level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]))
                print(f'1. i = {i} j = {j} level_num {level_num}')
            except TypeError:
                level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]))
                print("problem occurred with: ", i, j, flush=True)
        testProobs.append(probs[i][np.argsort(probs[i])][::-1][:1])
        print(f'2. i = {i} j = {j} testProobs {testProobs}')
    
    print(f'len(testProobs)  {len(testProobs)}')
    test_new_proobs = []
    for i in range(len(testProobs)):
        test_new_proobs.append(testProobs[i].tolist())
    test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]
    print(f'test_new_proobs {test_new_proobs}')
    return

In [68]:
proc_append()

len(preds)  50
1. i = 0 j = 0 level_num [0]
2. i = 0 j = 0 testProobs [array([0.99996457])]
1. i = 1 j = 0 level_num [0, 7]
2. i = 1 j = 0 testProobs [array([0.99996457]), array([0.90898802])]
1. i = 2 j = 0 level_num [0, 7, 1]
2. i = 2 j = 0 testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784])]
1. i = 3 j = 0 level_num [0, 7, 1, 1]
2. i = 3 j = 0 testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172])]
1. i = 4 j = 0 level_num [0, 7, 1, 1, 0]
2. i = 4 j = 0 testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172]), array([0.99828899])]
1. i = 5 j = 0 level_num [0, 7, 1, 1, 0, 0]
2. i = 5 j = 0 testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172]), array([0.99828899]), array([0.9999203])]
1. i = 6 j = 0 level_num [0, 7, 1, 1, 0, 0, 1]
2. i = 6 j = 0 testProobs [array([0.99996457]), array([0.90898802]), array([0.97083784]), array([0.99946172]), array(

In [57]:
def proc_append_print():
    i =1 
    j =1
    print (f' len (probs[i]) = {len (probs[i])}')
    x0 = [np.argsort(probs[i])]
    print (f' probs[i] {probs[i]}   x0, {x0}')
    x1 = probs[i][np.argsort(probs[i])]
    print (f' x1 = probs[i][np.argsort(probs[i])] {x1}')
    x2 = x1[::-1][:3] 
    print (f' x2 =  x1[::-1][:3]   {x2 } ') 
    print (f' x3 =  x2[j]   {x2[j]}   x2[j][0]   {x2[j][0] }')   
    x4 = probs[i][np.argsort(probs[i])][::-1][:3][j][0]
    print (f'  x4, {x4}')
    x5 = np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]
    x6 = np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]
    print (f'  x5, {x5}')
    print (f'  x6, {x6}')
    return    

In [58]:
 proc_append_print()

 len (probs[i]) = 8
 probs[i] [6.48720295e-04 8.92620466e-02 4.34744724e-04 1.66332553e-04
 1.66329314e-04 1.70206130e-04 1.63603376e-04 9.08988017e-01]   x0, [array([6, 4, 3, 5, 2, 0, 1, 7], dtype=int64)]
 x1 = probs[i][np.argsort(probs[i])] [1.63603376e-04 1.66329314e-04 1.66332553e-04 1.70206130e-04
 4.34744724e-04 6.48720295e-04 8.92620466e-02 9.08988017e-01]
 x2 =  x1[::-1][:3]   [9.08988017e-01 8.92620466e-02 6.48720295e-04] 


IndexError: invalid index to scalar variable.

In [98]:
def proc_save(level__1, filenamee):
    print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
    print("1)", len(A_test + A_train))
    print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
    print("3)", len(B_test + B_train))
    print("4)", len(testLevels + B_train))
    print("5)", len(testLevelss + trainLevelss))
    print("6)", int(len(test_new_proobs + train_new_proobs)))
    
    data = {
        'GTIN' : A_test + A_train,
        'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
        'Actual Level 2' : B_test + B_train,
        'Actuals' : testLevels + B_train,
        'Predictions' : testLevelss + trainLevelss,
        'Scores' : test_new_proobs + train_new_proobs,
        'L1 Name' : level__1
    }
    
    df = pd.DataFrame(data)
    df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
        'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
        'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
    df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
        'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
        'VND_ECOM_DSC', 'Source', 'Actual Level 2', 'Actuals', 'Predictions', 'Scores', 'L1 Name']]
    df.to_csv(DBFO + filenamee + '.csv', index=False)
    print(f"{filenamee}.csv   --file created successfully")
    return

In [119]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from pycaret.classification import setup
from pycaret.classification import compare_models
from pycaret.classification import tune_model
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
import re

In [5]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [101]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()

syndigo_mapped.shape (259085, 24)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 259085 entries, 0 to 259084
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   GTIN              259085 non-null  float64
 1   Level 1           257953 non-null  object 
 2   Level 2           256935 non-null  object 
 3   Level 3           247350 non-null  object 
 4   Level 4           179389 non-null  object 
 5   Level 5           45755 non-null   object 
 6   Level 6           5286 non-null    object 
 7   Level 7           1542 non-null    object 
 8   Level 8           179 non-null     object 
 9   Level 9           2 non-null       object 
 10  Level 10          0 non-null       object 
 11  ITM_ID            259069 non-null  float64
 12  PMY_DPT_CD        258915 non-null  float64
 13  PMY_DPT_DSC       258915 non-null  object 
 14  REC_DPT_CD        258915 non-null  object 
 15  REC_DPT_DSC       258915 non-null 

In [90]:

#vect = TfidfVectorizer(ngram_range= (1,2), max_features = 100000)
#X_subset, y_subset, subset_df =  prepare_data_2("Toys / Games / Hobbies") 
X_subset, y_subset, subset_df, level2_id_map =  prepare_data_2("Hospitality Supplies")

1. type(subset_df)= <class 'pandas.core.frame.DataFrame'>, len(subset_df)= 496 subset_df.shape (496, 24)  
 subset_df.head               GTIN               Level 1  \
2039  9.235249e+10  Hospitality Supplies   
3212  6.286897e+11  Hospitality Supplies   
3256  9.235292e+10  Hospitality Supplies   
4331  9.235241e+10  Hospitality Supplies   
5587  9.235243e+10  Hospitality Supplies   

                                    Level 2 Level 3 Level 4 Level 5 Level 6  \
2039  Paper / Plastic / Foam Plates & Bowls     NaN     NaN     NaN     NaN   
3212            Paper / Plastic / Foam Cups     NaN     NaN     NaN     NaN   
3256                       Plastic Flatware     NaN     NaN     NaN     NaN   
4331                       Plastic Flatware     NaN     NaN     NaN     NaN   
5587                       Plastic Flatware     NaN     NaN     NaN     NaN   

     Level 7 Level 8 Level 9  ... REC_DPT_CD    REC_DPT_DSC  DPT_CD  \
2039     NaN     NaN     NaN  ...         03            HBC    76.

In [87]:
level__1 = "Hospitality Supplies"
filenamee = "HOSPITALITY_SUPPLIES"
preds, preds_lrtf, probs, preds_train, probs_train = proc_predict(level__1)
   

Training - "Hospitality Supplies"
type(preds) <class 'numpy.ndarray'> len(preds) 50 
 preds[0:10] [0 7 1 1 0 0 1 1 1 0]
type(probs) <class 'numpy.ndarray'> len(probs) 50 
 probs[0:10] [[9.99964569e-01 1.10869811e-05 9.28915670e-06 2.90611540e-06
  2.97634138e-06 2.99779565e-06 2.82694203e-06 3.34749880e-06]
 [6.49287420e-04 8.92135251e-02 4.34854418e-04 1.66365035e-04
  1.66066905e-04 1.70142269e-04 1.63521026e-04 9.09036238e-01]
 [1.47153211e-02 9.70833942e-01 9.53746369e-03 6.93434187e-04
  1.12437008e-03 1.08721657e-03 1.20039911e-03 8.07853118e-04]
 [7.17467419e-05 9.99461348e-01 3.16981222e-05 2.02860547e-05
  1.94752005e-05 2.08325893e-05 2.02444738e-05 3.54368357e-04]
 [9.98290748e-01 1.14870360e-03 2.93146336e-04 5.60808581e-05
  5.64360190e-05 5.68049291e-05 5.56012399e-05 4.24790211e-05]
 [9.99920116e-01 5.08096841e-05 1.03869392e-05 3.37059063e-06
  5.80910769e-06 3.40816898e-06 3.38925760e-06 2.71015256e-06]
 [2.21784669e-01 6.77404486e-01 6.26859837e-02 8.38881982e-03
  1.

In [102]:
proc_save(level__1, filenamee) 

---------------------------------------------\FINAL COUNTS:
---------------------------------------------
1) 493
2) 493
3) 493
4) 493
5) 493
6) 493
HOSPITALITY_SUPPLIES.csv   --file created successfully


## Wip

In [ ]:
 x.dropna().unique()

In [112]:

#key = syndigo_mapped['Level 1'].unique()
key = unique_no_nan(syndigo_mapped['Level 1'])
key
 

array(['Food / Beverages', 'Health & Beauty', 'Beer / Wine / Spirits',
       'Kitchen & Bathroom', 'Cleaning & Janitorial',
       'Home & Venue Decoration', 'Livestock & Pet Supplies',
       'Toys / Games / Hobbies', 'Childcare', 'Tobacco Products',
       'Apparel', 'Gardening & Outdoors',
       'Sports & Outdoor Recreation Equipment', 'Office Supplies',
       'Plumbing & Water Service', 'Lighting & Fans', 'Appliances',
       'Electronics', 'Not classified', 'Automotive', 'Tools', 'Hardware',
       'Books & Videos', 'Arts & Crafts', 'Electrical',
       'Heating / Ventilation / Air Conditioning', 'Hospitality Supplies',
       'Furniture', 'Power Sports', 'Power Transmission & Motion Control',
       'Marine', 'Flooring', 'Paints & Coatings', 'Building Supplies',
       'Musical Instruments', 'Material Handling',
       'Agricultural Equipment'], dtype=object)

In [127]:
#val1 = list(map( re.sub('&', ''),  key))
#a = map(lambda x : int(x) , list(np.zeros(N)))
val1 = list( map(lambda x : re.sub('&|/', '', x), key))
#string = re.sub("m|n|\s", "#", string)
val1 = list( map(lambda x : re.sub(' +', '_', x), val1))
val1

['Food_Beverages',
 'Health_Beauty',
 'Beer_Wine_Spirits',
 'Kitchen_Bathroom',
 'Cleaning_Janitorial',
 'Home_Venue_Decoration',
 'Livestock_Pet_Supplies',
 'Toys_Games_Hobbies',
 'Childcare',
 'Tobacco_Products',
 'Apparel',
 'Gardening_Outdoors',
 'Sports_Outdoor_Recreation_Equipment',
 'Office_Supplies',
 'Plumbing_Water_Service',
 'Lighting_Fans',
 'Appliances',
 'Electronics',
 'Not_classified',
 'Automotive',
 'Tools',
 'Hardware',
 'Books_Videos',
 'Arts_Crafts',
 'Electrical',
 'Heating_Ventilation_Air_Conditioning',
 'Hospitality_Supplies',
 'Furniture',
 'Power_Sports',
 'Power_Transmission_Motion_Control',
 'Marine',
 'Flooring',
 'Paints_Coatings',
 'Building_Supplies',
 'Musical_Instruments',
 'Material_Handling',
 'Agricultural_Equipment']

In [129]:
#values = unique_no_nan(syndigo_mapped['Level 1']).re.sub('&', '')
#values = lower(unique_no_nan(syndigo_mapped['Level 1']))
values= list(map(str.lower, key))
values

['food / beverages',
 'health & beauty',
 'beer / wine / spirits',
 'kitchen & bathroom',
 'cleaning & janitorial',
 'home & venue decoration',
 'livestock & pet supplies',
 'toys / games / hobbies',
 'childcare',
 'tobacco products',
 'apparel',
 'gardening & outdoors',
 'sports & outdoor recreation equipment',
 'office supplies',
 'plumbing & water service',
 'lighting & fans',
 'appliances',
 'electronics',
 'not classified',
 'automotive',
 'tools',
 'hardware',
 'books & videos',
 'arts & crafts',
 'electrical',
 'heating / ventilation / air conditioning',
 'hospitality supplies',
 'furniture',
 'power sports',
 'power transmission & motion control',
 'marine',
 'flooring',
 'paints & coatings',
 'building supplies',
 'musical instruments',
 'material handling',
 'agricultural equipment']

In [21]:
# conda install -c conda-forge pycaret

# setup the dataset
#grid = setup(data=X_subset, target= y_subset, html=False, silent=True, verbose=False)
grid = setup(data=df3, target= df3.columns[-1], html=False, verbose=False)
# evaluate models and compare models
best = compare_models()
# report the best model
print(best)

                                    Model  Accuracy     AUC  Recall   Prec.  \
ridge                    Ridge Classifier    0.7743  0.0000  0.7743  0.7697   
svm                   SVM - Linear Kernel    0.7698  0.0000  0.7698  0.7654   
et                 Extra Trees Classifier    0.7667  0.7501  0.7667  0.7609   
rf               Random Forest Classifier    0.7591  0.7502  0.7591  0.7541   
gbc          Gradient Boosting Classifier    0.7497  0.7454  0.7497  0.7540   
knn                K Neighbors Classifier    0.7416  0.7256  0.7416  0.7370   
dt               Decision Tree Classifier    0.7322  0.6641  0.7322  0.7299   
lr                    Logistic Regression    0.7286  0.7516  0.7286  0.7326   
lightgbm  Light Gradient Boosting Machine    0.7067  0.7339  0.7067  0.6991   
nb                            Naive Bayes    0.6565  0.6301  0.6565  0.6626   
ada                  Ada Boost Classifier    0.6234  0.4970  0.6234  0.5346   
lda          Linear Discriminant Analysis    0.4935 

In [24]:
# tune model hyperparameters
best_yune = tune_model( RidgeClassifier(), n_iter=200, choose_better=True)
# report the best model
print(best_tune)


Processing:  14%|██████████▎                                                             | 1/7 [00:00<00:02,  2.75it/s]

Fitting 10 folds for each of 200 candidates, totalling 2000 fits



Processing: 100%|███████████████████████████████████████████████████████████████████████| 7/7 [41:02<00:00, 224.00s/it]
                                                                                                                       

      Accuracy  AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                       
0       0.7723  0.0  0.7723  0.7708  0.7598  0.7098  0.7131
1       0.7902  0.0  0.7902  0.7777  0.7777  0.7331  0.7355
2       0.7411  0.0  0.7411  0.7254  0.7301  0.6703  0.6724
3       0.7758  0.0  0.7758  0.7718  0.7647  0.7114  0.7158
4       0.7444  0.0  0.7444  0.7328  0.7279  0.6701  0.6756
5       0.7848  0.0  0.7848  0.7654  0.7668  0.7238  0.7277
6       0.8027  0.0  0.8027  0.8037  0.7922  0.7481  0.7511
7       0.8117  0.0  0.8117  0.7968  0.7986  0.7597  0.7621
8       0.8027  0.0  0.8027  0.8076  0.7930  0.7473  0.7520
9       0.7713  0.0  0.7713  0.7731  0.7619  0.7072  0.7119
Mean    0.7797  0.0  0.7797  0.7725  0.7672  0.7181  0.7217
Std     0.0226  0.0  0.0226  0.0258  0.0232  0.0293  0.0290


NameError: name 'best_tune' is not defined

In [25]:
print(best_yune)

RidgeClassifier(alpha=0.47, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=None, solver='auto',
                tol=0.0001)


In [16]:
df_y.value_counts( dropna = False)

1    1139
0     540
5     394
3     372
6     258
7     191
4     164
8      82
2      38
9      12
Name: Level 2, dtype: int64

In [27]:
# tune model hyperparameters
best_tune = tune_model( LogisticRegressionCV(), n_iter=200, choose_better=True)
# report the best model
print(best_tune)



Processing:   0%|                                                                                | 0/7 [00:00<?, ?it/s]



Fitting 10 folds for each of 200 candidates, totalling 2000 fits


ValueError: Invalid parameter 'actual_estimator' for estimator LogisticRegressionCV(Cs=10, class_weight='balanced', cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0). Valid parameters are: ['Cs', 'class_weight', 'cv', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratios', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'refit', 'scoring', 'solver', 'tol', 'verbose'].

In [13]:
y_subset.index

Int64Index([    53,     71,     80,    108,    170,    180,    205,    290,
               297,    361,
            ...
            257772, 257788, 257916, 257958, 257985, 258215, 258376, 258717,
            258827, 258831],
           dtype='int64', length=3190)

In [66]:
df1=dfx.reset_index().drop('index', axis = 1) 
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,16564,16565,16566,16567,16568,16569,16570,16571,16572,16573
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
df2=dfy.reset_index().drop('index', axis = 1)

df2.head()

,Level 2
0,Games & Puzzles
1,Toy Figures
2,Toy Figures
3,Games & Puzzles
4,Toy Figures


In [67]:
df3 = pd.merge(df1, df2, left_index=True, right_index=True)
df3.head()

,0,1,2,3,4,5,6,7,8,9,...,16565,16566,16567,16568,16569,16570,16571,16572,16573,Level 2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Games & Puzzles
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Games & Puzzles
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures


In [68]:
df3['Level 2'].value_counts(dropna = False)

Toy Figures                    1139
Games & Puzzles                 540
Construction & Modeling         394
Baby Toys & Games               372
Toy Vehicles                    258
Outdoor Toys & Games            191
Role Playing & Pretend Play     164
Sports Toys & Games              82
Remote & Radio Control           38
Nature / Science                 12
Name: Level 2, dtype: int64

In [57]:
dfx = pd.DataFrame(X_subset.toarray())
#dfx['y'] = y_subset
subset_df[['Level 2']]
dfy =  subset_df[['Level 2']]
dfz = pd.merge(dfx, dfy, left_index=True, right_index=True)
dfz.head()

,0,1,2,3,4,5,6,7,8,9,...,16565,16566,16567,16568,16569,16570,16571,16572,16573,Level 2
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Games & Puzzles
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Games & Puzzles
170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures


In [61]:
dfz['Level 2'].value_counts(dropna = True)

Toy Figures                    22
Games & Puzzles                13
Construction & Modeling         6
Baby Toys & Games               5
Role Playing & Pretend Play     4
Toy Vehicles                    2
Remote & Radio Control          1
Outdoor Toys & Games            1
Name: Level 2, dtype: int64

In [ ]:
subset_df.

In [56]:
subset_df.head(10)

,GTIN,Level 1,Level 2,Level 3,Level 4,Level 5,Level 6,Level 7,Level 8,Level 9,...,REC_DPT_CD,REC_DPT_DSC,DPT_CD,DPT_DSC,COM_CD,COM_DSC,SUBCOM_CD,SUBCOM_DSC,VND_ECOM_DSC,ITEM_SUBCOM_text
53,6.305095e+11,Toys / Games / Hobbies,Games & Puzzles,Board Games,Family,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,842.0,GAMES,87608.0,BOARD GAMES,Monopoly® Board Game,monopoly® board game board games
71,6.305095e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,973.0,BASIC GIRL,87662.0,FASHION DOLLS,DPR TANGLED STORY FIG,dpr tangled story fig fashion dolls
80,8.879616e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,954.0,BASIC PRE-SCHOOL,87651.0,FISHER PRICE,Fisher-Price Imaginext Jurassic World Dinosaur...,fisher-price imaginext jurassic world dinosaur...
108,6.894230e+11,Toys / Games / Hobbies,Games & Puzzles,Spinning Tops,NaN,NaN,NaN,NaN,NaN,NaN,...,01,GROCERY,1.0,GROC-ALL OTHER,135.0,KOSHER FOODS AND PRODUCTS,18278.0,KOSHER CANDY,GEDILLA CANDY - CHANUKAH DREIDLE,gedilla candy - chanukah dreidle kosher candy
170,8.879616e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Sets,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,973.0,BASIC GIRL,97305.0,BARBIE DOLLS,MTL BRB LIC FASHION LOOKS,mtl brb lic fashion looks barbie dolls
180,6.305096e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,834.0,BASIC BOYS TOYS,87605.0,ACTION FIGURES,SPD 6 QUICK SHOT,spd 6 quick shot action figures
205,6.535700e+11,Toys / Games / Hobbies,Games & Puzzles,Puzzles,Brain Teasers,Assembly / Disassembly,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,842.0,GAMES,87614.0,CARD GAMES,Hasbro Gaming Rubik's Cube Game,hasbro gaming rubik's cube game card games
290,6.305096e+11,Toys / Games / Hobbies,Toy Figures,Stuffed Animals,NaN,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,954.0,BASIC PRE-SCHOOL,87646.0,PRESCHOOL TOYS,Hasbro Sesame Street Abby Cadabby Mini Plush Doll,hasbro sesame street abby cadabby mini plush d...
297,8.879616e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Sets,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,834.0,BASIC BOYS TOYS,87668.0,CONSTRUCTION,MEGA Mega Construx Pokmon Blastoise,mega mega construx pokmon blastoise construction
361,6.535696e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,834.0,BASIC BOYS TOYS,87605.0,ACTION FIGURES,TRA3 CYBERTECH DLX,tra3 cybertech dlx action figures


In [53]:
subset_df['Level 2']

53                Games & Puzzles
71                    Toy Figures
80                    Toy Figures
108               Games & Puzzles
170                   Toy Figures
                   ...           
258215    Construction & Modeling
258376               Toy Vehicles
258717                Toy Figures
258827                Toy Figures
258831    Construction & Modeling
Name: Level 2, Length: 3190, dtype: object

In [54]:
subset_df[['Level 2']]

,Level 2
53,Games & Puzzles
71,Toy Figures
80,Toy Figures
108,Games & Puzzles
170,Toy Figures
...,...
258215,Construction & Modeling
258376,Toy Vehicles
258717,Toy Figures
258827,Toy Figures


In [55]:
subset_df[['Level 2']].index

Int64Index([    53,     71,     80,    108,    170,    180,    205,    290,
               297,    361,
            ...
            257772, 257788, 257916, 257958, 257985, 258215, 258376, 258717,
            258827, 258831],
           dtype='int64', length=3190)

In [14]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [21]:
models = []
models.append(('LRC', LogisticRegressionCV(solver='saga', multi_class='multinomial')))
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_subset, y_subset, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


KeyboardInterrupt: 

In [69]:

# conda install -c conda-forge pycaret
from pycaret.classification import setup
from pycaret.classification import compare_models

# setup the dataset
#grid = setup(data=X_subset, target= y_subset, html=False, silent=True, verbose=False)
grid = setup(data=df3, target= df3.columns[-1], html=False, verbose=False)
# evaluate models and compare models
best = compare_models()
# report the best model
print(best)

NameError: name 'best' is not defined

In [70]:
# evaluate models and compare models
best = compare_models()
# report the best model
print(best)

                                    Model  Accuracy     AUC  Recall   Prec.  \
svm                   SVM - Linear Kernel    0.7631  0.0000  0.7631  0.7556   
ridge                    Ridge Classifier    0.7590  0.0000  0.7590  0.7483   
et                 Extra Trees Classifier    0.7510  0.7434  0.7510  0.7393   
rf               Random Forest Classifier    0.7488  0.7444  0.7488  0.7356   
gbc          Gradient Boosting Classifier    0.7398  0.7425  0.7398  0.7355   
lr                    Logistic Regression    0.7254  0.7502  0.7254  0.7260   
dt               Decision Tree Classifier    0.7125  0.6556  0.7125  0.7060   
knn                K Neighbors Classifier    0.7111  0.7123  0.7111  0.6986   
lightgbm  Light Gradient Boosting Machine    0.6955  0.7302  0.6955  0.6844   
nb                            Naive Bayes    0.6574  0.6323  0.6574  0.6660   
ada                  Ada Boost Classifier    0.6498  0.5107  0.6498  0.5892   
lda          Linear Discriminant Analysis    0.4250 

In [ ]:
# tune model hyperparameters
best_yune = tune_model(SGDClassifier(), n_iter=200, choose_better=True)
# report the best model
print(best_tune)


Processing:  14%|██████████▎                                                             | 1/7 [00:00<00:04,  1.48it/s]

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


In [19]:
#getnnz(X_subset)
#len(X_subset) # NW; sparse matrix length is ambiguous; use getnnz() or shape[0]
X_subset.shape[0] # 3190

3190

In [47]:
subset_df['Level 2'].value_counts(dropna = True)

Toy Figures                    1139
Games & Puzzles                 540
Construction & Modeling         394
Baby Toys & Games               372
Toy Vehicles                    258
Outdoor Toys & Games            191
Role Playing & Pretend Play     164
Sports Toys & Games              82
Remote & Radio Control           38
Nature / Science                 12
Name: Level 2, dtype: int64

In [32]:
type(y_subset)
y_subset.shape[0]

3190

In [30]:
X_subset.toarray().shape

(3190, 16574)

In [36]:
X_subset.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
dfy = y_subset.reset_index()

,index,Level 2
0,53,0
1,71,1
2,80,1
3,108,0
4,170,1
...,...,...
3185,258215,5
3186,258376,6
3187,258717,1
3188,258827,1


In [34]:
y_subset[0:10]

53     0
71     1
80     1
108    0
170    1
180    1
205    0
290    1
297    1
361    1
Name: Level 2, dtype: int64

In [3]:
# Reading PIMMART data
pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
    pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)

# Reading Syndigo 259K data
synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
    synd_ALL[i] = synd_ALL[i].astype(np.float64)

# Stripping spaces from all columns
df_obj = synd_ALL.select_dtypes(['object'])
synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [4]:
syndigo_mapped = synd_ALL
pimmart = pim_gtin_mapped

In [5]:
syndigo_mapped.drop_duplicates('GTIN', inplace = True)

In [6]:
syndigo_mapped['ITEM_SUBCOM_text'] = \
(syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()

In [7]:
syndigo_mapped['Level 1'].value_counts()

Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
Not classified                                1018
Hardware                                       923
Arts & Crafts                                  803
Automotive                                     778
Appliances                     

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range= (1,2), max_features = 800000)

In [9]:
syndigo_mapped['Level 1'].value_counts()

Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
Not classified                                1018
Hardware                                       923
Arts & Crafts                                  803
Automotive                                     778
Appliances                     

## RUN UNTIL HERE only ONCE

In [10]:
#level__1 = "Toys / Games / Hobbies"
#filenamee = "TOYS_GAMES_HOBBIES"

# level__1 = "Tobacco Products"
# filenamee = "TOBACCO_PRODUCTS"

# level__1 = "Cleaning & Janitorial"
# filenamee = "CLEANING_JANITORIAL"

# level__1 = "Office Supplies"
# filenamee = "OFFICE_SUPPLIES"

# level__1 = "Beer / Wine / Spirits"
# filenamee = "BEER_WINE_SPIRITS"

# level__1 = "Livestock & Pet Supplies"
# filenamee = "LIVESTOCK_PET_SUPPLIES"

# level__1 = "Kitchen & Bathroom"
# filenamee = "KITCHEN_BATHROOM"

level__1 = "Hospitality Supplies"
filenamee = "HOSPITALITY_SUPPLIES"

# level__1 = "Home & Venue Decoration"
# filenamee = "HOME_VENUE_DECORATION"

# level__1 = "Heating / Ventilation / Air Conditioning"
# filenamee = "HEATING_VENTILATION_AIR_CONDITIONING"

# level__1 = "Gardening & Outdoors"
# filenamee = "GARDENING_OUTDOORS"

# level__1 = "Sports & Outdoor Recreation Equipment"
# filenamee = "SPORTS_OUTDOOR_RECREATION_EQUIPMENT"

syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)

Note Pads & Paper                          1
Beverage Carriers                          1
Beverage Container Covers                  1
Dispensers                                 3
Comment Cards                              3
Ice Buckets                                4
Food & Beverage Condiments                 5
Paper / Plastic / Foam Cups Sleeves        5
Plastic Flatware                          65
Paper / Plastic / Foam Cups              138
Paper / Plastic / Foam Plates & Bowls    270
Name: Level 2, dtype: int64

In [11]:
level_1 = level__1
subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
series = syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)
if series.tolist():
    subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series==1].index.tolist())].index)

vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 1000000)
X_subset = vect_subset.fit_transform(subset_df.ITEM_SUBCOM_text)
level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
y_subset  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
A_train, A_test, B_train, B_test = train_test_split(subset_df.GTIN.tolist(), subset_df['Level 2'].tolist(), test_size=0.1, stratify=y_subset, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size= 0.1, stratify=y_subset, random_state=42)
lr_tf = LogisticRegressionCV(cv = 5, multi_class = 'multinomial', solver = 'saga', n_jobs=-1, max_iter=10000000)

print(f"Training - \"{level_1}\"")
lr_tf.fit(X_train, y_train)
preds = lr_tf.predict(X_test)
preds_lrtf = preds
probs = lr_tf.predict_proba(X_test)
preds_train = lr_tf.predict(X_train)
probs_train = lr_tf.predict_proba(X_train)
print(f"Done Training - \"{level_1}\"")

Training - "Hospitality Supplies"


C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Done Training - "Hospitality Supplies"


In [12]:
level_num = []
testProobs = []
number = 1

print(len(preds))
for i in range(len(preds)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    testProobs.append(probs[i][np.argsort(probs[i])][::-1][:1])

test_new_proobs = []
for i in range(len(testProobs)):
    test_new_proobs.append(testProobs[i].tolist())
test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]

50


In [13]:
level_num = []
trainProobs = []

print(len(preds_train))
for i in range(len(preds_train)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    trainProobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1]) 

train_new_proobs = []
for i in range(len(trainProobs)):
    train_new_proobs.append(trainProobs[i].tolist())
train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]

443


In [14]:
testLevels = []
for j in y_test:
    testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])

testLevelss = []
for j in preds:
    testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

trainLevelss = []
for j in preds_train:
    trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

In [15]:
print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
print("1)", len(A_test + A_train))
print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
print("3)", len(B_test + B_train))
print("4)", len(testLevels + B_train))
print("5)", len(testLevelss + trainLevelss))
print("6)", int(len(test_new_proobs + train_new_proobs)))

---------------------------------------------\FINAL COUNTS:
---------------------------------------------
1) 493
2) 493
3) 493
4) 493
5) 493
6) 493


In [16]:
data = {
    'GTIN' : A_test + A_train,
    'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
    'Actual Level 2' : B_test + B_train,
    'Actuals' : testLevels + B_train,
    'Predictions' : testLevelss + trainLevelss,
    'Scores' : test_new_proobs + train_new_proobs,
    'L1 Name' : level_1
}

df = pd.DataFrame(data)
df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
    'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
    'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
    'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
    'VND_ECOM_DSC', 'Source', 'Actual Level 2', 'Actuals', 'Predictions', 'Scores', 'L1 Name']]

## Keep the output of the below cell for all Level 1 executed

In [17]:
df.to_csv(DBFO + filenamee + '.csv', index=False)
print(f"{filenamee}.csv   --file created successfully")

HOSPITALITY_SUPPLIES.csv   --file created successfully
